In [1]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 42.1 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 89.0 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 180.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 164.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 113.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 48.6 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 138.4 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 149.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 135.6 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 134.4 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 138.3 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm # Use tqdm.notebook for Jupyter progress bars

# --- PyTorch Imports for ML Model ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Determine if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# --- ML Model Configuration ---
WINDOW_SIZE = 150       # Number of time steps (data points) in each input sequence/window
STRIDE = 5             # How many steps to move the window (1 means maximum overlap, covering every point)
HIDDEN_SIZE = 64
LATENT_SIZE = 32
EPOCHS = 50            # Number of training epochs for the autoencoder
BATCH_SIZE = 64        # Batch size for training
ANOMALY_THRESHOLD_QUANTILE = 0.70 # Quantile of reconstruction errors on NORMAL data to set anomaly threshold


# --- Post-processing configuration (applied to ML predictions) ---
MERGE_GAP_SECONDS = 3             # Maximum time gap between predicted wake segments to merge them
MIN_WAKE_DURATION_SECONDS = 0.5     # Minimum duration for a detected event to be considered a wake


# --- Global paths ---
base_data_dir = 'processed_ts'
output_base_dir_ml = 'detected_wakes_LSTM_AE' # New output directory for ML predictions
plots_output_dir_ml = 'detected_wakes_LSTM_AE/wake_plots/w150,s5,th70'         # New directory for ML plots
model_save_dir = 'detected_wakes_LSTM_AE/saved_models/w150,s5'

In [5]:
def load_and_preprocess_data(file_path):
    """
    Loads the CSV data and performs basic preprocessing.
    Ensures 't_s' is sorted and resets index.
    """
    try:
        df = pd.read_csv(file_path)
        df = df.sort_values(by='t_s').reset_index(drop=True)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error loading or preprocessing data from {file_path}: {e}")
        return None

In [6]:
def get_ground_truth_wakes(df):
    """
    Extracts ground truth wake intervals from the 'wake_label' column.
    Returns a list of (start_time, end_time) tuples.
    """
    ground_truth_wakes = []
    in_wake = False
    wake_start_time = None

    for i, row in df.iterrows():
        t_s = row['t_s']
        wake_label = row['wake_label']

        if wake_label == 1 and not in_wake:
            wake_start_time = t_s
            in_wake = True
        elif wake_label == 0 and in_wake:
            wake_end_time = t_s
            ground_truth_wakes.append((wake_start_time, wake_end_time))
            in_wake = False
    
    # Handle case where wake extends to the end of the time series
    if in_wake:
        ground_truth_wakes.append((wake_start_time, df['t_s'].iloc[-1]))

    return ground_truth_wakes

In [7]:
def calculate_iou(interval1, interval2):
    """
    Calculates the Intersection over Union (IoU) of two time intervals.
    Intervals are (start, end) tuples.
    """
    start1, end1 = interval1
    start2, end2 = interval2

    # Calculate intersection duration
    intersection_start = max(start1, start2)
    intersection_end = min(end1, end2)
    
    intersection_duration = max(0, intersection_end - intersection_start)

    # Calculate union duration
    union_duration = (max(end1, end2) - min(start1, start2))

    if union_duration == 0:
        return 0.0 # No union means no overlap possible
    
    return intersection_duration / union_duration

In [8]:
def evaluate_detection(ground_truth_wakes, predicted_wakes, iou_threshold=0.5):
    """
    Evaluates the detection performance based on IoU.
    Returns TP, FP, FN, Accuracy, Precision, Recall, F1-Score.
    """
    tp = 0
    fp = 0
    fn = 0

    # Sets to keep track of matched ground truth and used predicted wakes
    matched_gt_indices = set()
    used_pred_indices = set()

    # Iterate through predicted wakes to find the best ground truth match for each
    for pred_idx, pred_wake in enumerate(predicted_wakes):
        best_iou_for_pred = 0.0
        potential_gt_idx_for_pred = -1

        for gt_idx, gt_wake in enumerate(ground_truth_wakes):
            if gt_idx in matched_gt_indices: # If this GT wake is already matched, skip it
                continue

            iou = calculate_iou(gt_wake, pred_wake)
            if iou > best_iou_for_pred:
                best_iou_for_pred = iou
                potential_gt_idx_for_pred = gt_idx
        
        # If the best IoU for this predicted wake is above threshold AND it matched an unmatched GT wake
        if best_iou_for_pred >= iou_threshold and potential_gt_idx_for_pred != -1:
            tp += 1
            matched_gt_indices.add(potential_gt_idx_for_pred) # Mark GT wake as matched
            used_pred_indices.add(pred_idx) # Mark predicted wake as used for a TP
        else:
            fp += 1 # This predicted wake is a False Positive

    # Calculate False Negatives: any ground truth wake that was not matched
    fn = len(ground_truth_wakes) - len(matched_gt_indices)

    total_ground_truth = len(ground_truth_wakes)
    
    # Calculate performance metrics
    accuracy = tp / total_ground_truth if total_ground_truth > 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    # F1-Score is the harmonic mean of precision and recall
    if (precision + recall) == 0:
        f1_score = 0.0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    
    return tp, fp, fn, accuracy, precision, recall, f1_score

In [9]:
def plot_wake_detection(df, file_name, save_dir=None, recon_col: str | None = "z_m_recon"):
    """
    Plots the time series with GT (red spans) and predicted (green spans).
    If recon_col is present in df, overlays the reconstructed signal as a line.
    """
    plt.figure(figsize=(15, 6))

    # Original signal
    plt.plot(df['t_s'], df['z_m'], color='black', linewidth=0.8, label='z_m (Vertical Displacement)')

    # (NEW) Reconstructed signal overlay if available
    if recon_col is not None and recon_col in df.columns:
        plt.plot(df['t_s'], df[recon_col], linewidth=0.9, label='Reconstruction')

    # --- Ground truth spans (red) ---
    in_gt_wake = False
    gt_wake_start = None
    gt_label_added = False
    for _, row in df.iterrows():
        if row['wake_label'] == 1 and not in_gt_wake:
            gt_wake_start = row['t_s']; in_gt_wake = True
        elif row['wake_label'] == 0 and in_gt_wake:
            plt.axvspan(gt_wake_start, row['t_s'], color='red', alpha=0.3,
                        label='Ground Truth Wake' if not gt_label_added else "")
            gt_label_added = True; in_gt_wake = False
    if in_gt_wake:
        plt.axvspan(gt_wake_start, df['t_s'].iloc[-1], color='red', alpha=0.3,
                    label='Ground Truth Wake' if not gt_label_added else "")

    # --- Predicted spans (green) ---
    in_pred_wake = False
    pred_wake_start = None
    pred_label_added = False
    if 'predicted_wake_label' in df.columns:
        for _, row in df.iterrows():
            if row['predicted_wake_label'] == 1 and not in_pred_wake:
                pred_wake_start = row['t_s']; in_pred_wake = True
            elif row['predicted_wake_label'] == 0 and in_pred_wake:
                plt.axvspan(pred_wake_start, row['t_s'], color='green', alpha=0.3,
                            label='Predicted Wake' if not pred_label_added else "")
                pred_label_added = True; in_pred_wake = False
        if in_pred_wake:
            plt.axvspan(pred_wake_start, df['t_s'].iloc[-1], color='green', alpha=0.3,
                        label='Predicted Wake' if not pred_label_added else "")

    plt.title(f'Wake Detection for {file_name}', fontsize=16)
    plt.xlabel('Time (s)'); plt.ylabel('z_m (m)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.tight_layout()

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plot_path = os.path.join(save_dir, f'{os.path.basename(file_name).replace(".csv", "")}_wake_plot.png')
        plt.savefig(plot_path, dpi=300)
        plt.close()
    else:
        plt.show()


In [10]:
class TimeSeriesDataset(Dataset):
    """
    A custom PyTorch Dataset for time series sequences.
    """
    def __init__(self, sequences, original_indices):
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.original_indices = original_indices # Keep track of original start indices

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.original_indices[idx]

In [11]:
def create_sequences_pytorch(df_data, window_size, stride, scaler=None, fit_scaler=True):
    """
    Creates overlapping sequences (windows) from time series data ('z_m' column)
    and returns them as NumPy arrays along with original start indices.
    Manages StandardScaler fitting/transforming.
    """
    data = df_data['z_m'].values.reshape(-1, 1) # Reshape for StandardScaler

    if fit_scaler:
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)
    else:
        if scaler is None:
            raise ValueError("Scaler must be provided if fit_scaler is False.")
        scaled_data = scaler.transform(data)

    sequences = []
    original_indices = [] # Stores the starting index in the original DataFrame for each sequence
    """
    This list stores the starting index in the original scaled_data array for each created window.
    This is very important later when we need to map the model's predictions (which are per-window)
    back to the original time series data points.
    """
    
    # Loop to create sliding windows
    for i in range(0, len(scaled_data) - window_size + 1, stride):
        sequences.append(scaled_data[i : i + window_size])
        original_indices.append(i) # Record the start index of this window
    
    return np.array(sequences), np.array(original_indices), scaler

In [12]:
class LSTMAutoencoder(nn.Module):
    """
    A simple LSTM Autoencoder model in PyTorch.
    """
    def __init__(self, input_features, hidden_size=HIDDEN_SIZE, latent_size=LATENT_SIZE, window_size=WINDOW_SIZE):
        super(LSTMAutoencoder, self).__init__()
        self.window_size = window_size
        self.input_features = input_features

        # Encoder
        self.encoder_lstm1 = nn.LSTM(input_features, hidden_size, batch_first=True)
        self.encoder_lstm2 = nn.LSTM(hidden_size, latent_size, batch_first=True) # Last hidden state is latent representation

        # Decoder
        # Input to decoder is the repeated latent vector
        self.decoder_lstm1 = nn.LSTM(latent_size, hidden_size, batch_first=True)
        self.decoder_lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, input_features) # Output layer to match original feature size

    def forward(self, x):
        # Encoder
        # Pass through first LSTM, get all hidden states, we only need the last one (h_n, c_n)
        _, (h_n, _) = self.encoder_lstm1(x)
        # Pass through second LSTM, get only the last hidden state from the last layer (h_n)
        # h_n will be of shape (num_layers * num_directions, batch_size, hidden_size)
        # We want the last layer's hidden state, which is h_n[-1] if num_layers=1 for second LSTM
        _, (h_n2, _) = self.encoder_lstm2(h_n.permute(1, 0, 2)) # Permute to (batch_size, num_layers, hidden_size) for next LSTM
        
        # Latent representation is h_n2[-1] (last layer's last hidden state)
        latent_vector = h_n2[-1] # Shape: (batch_size, latent_size)

        # RepeatVector equivalent: repeat the latent vector for each time step of the decoder
        # latent_vector.unsqueeze(1) changes from (batch_size, latent_size) to (batch_size, 1, latent_size)
        # .repeat(1, self.window_size, 1) repeats it across the sequence length dimension
        decoder_input = latent_vector.unsqueeze(1).repeat(1, self.window_size, 1)

        # Decoder
        decoder_output1, _ = self.decoder_lstm1(decoder_input)
        decoder_output2, _ = self.decoder_lstm2(decoder_output1)
        
        # Apply linear layer to each time step using TimeDistributed equivalent
        # Reshape to (batch_size * window_size, hidden_size) for linear layer
        output = self.output_layer(decoder_output2.reshape(-1, decoder_output2.shape[-1]))
        # Reshape back to (batch_size, window_size, input_features)
        output = output.reshape(x.shape[0], self.window_size, self.input_features)

        return output

In [13]:
def get_predicted_wake_intervals_from_errors(df, original_indices, errors, anomaly_threshold, window_size, 
                                             merge_gap_seconds, min_wake_duration_seconds):
    """
    Converts window-level anomaly scores (reconstruction errors) into continuous wake intervals.
    It then populates a 'predicted_wake_label' column in the DataFrame and performs
    post-processing (merging close intervals, filtering by min duration).
    """
    df_copy = df.copy() 

    df_copy['predicted_wake_label'] = 0

    anomalous_window_indices_in_errors = np.where(errors > anomaly_threshold)[0]

    for error_idx in anomalous_window_indices_in_errors:
        start_df_idx = original_indices[error_idx] 
        end_df_idx = start_df_idx + window_size    
        
        end_df_idx = min(end_df_idx, len(df_copy))
        
        df_copy.loc[start_df_idx:end_df_idx-1, 'predicted_wake_label'] = 1

    predicted_wakes_raw = []
    in_wake_segment = False
    segment_start_time = None

    for i, row in df_copy.iterrows():
        t_s = row['t_s']
        predicted_label = row['predicted_wake_label']

        if predicted_label == 1 and not in_wake_segment:
            segment_start_time = t_s
            in_wake_segment = True
        elif predicted_label == 0 and in_wake_segment:
            segment_end_time = t_s
            predicted_wakes_raw.append((segment_start_time, segment_end_time))
            in_wake_segment = False
    
    if in_wake_segment:
        predicted_wakes_raw.append((segment_start_time, df_copy['t_s'].iloc[-1]))
    
    final_predicted_wakes = []
    if predicted_wakes_raw:
        predicted_wakes_raw.sort() 

        current_wake_start_time = predicted_wakes_raw[0][0]
        current_wake_end_time = predicted_wakes_raw[0][1]

        for i in range(1, len(predicted_wakes_raw)):
            next_segment_start_time = predicted_wakes_raw[i][0]
            next_segment_end_time = predicted_wakes_raw[i][1]

            if (next_segment_start_time - current_wake_end_time) <= merge_gap_seconds:
                current_wake_end_time = max(current_wake_end_time, next_segment_end_time)
            else:
                if (current_wake_end_time - current_wake_start_time) >= min_wake_duration_seconds:
                    final_predicted_wakes.append((current_wake_start_time, current_wake_end_time))
                current_wake_start_time = next_segment_start_time
                current_wake_end_time = next_segment_end_time
        
        if (current_wake_end_time - current_wake_start_time) >= min_wake_duration_seconds:
            final_predicted_wakes.append((current_wake_start_time, current_wake_end_time))

    return final_predicted_wakes, df_copy

In [14]:
def get_wake_labels_for_windows(df, original_indices, window_size):
    """
    Assigns a label to each window: 1 if ANY point in the window is a ground truth wake (label 1),
    0 otherwise (meaning the entire window is composed of normal points).
    """
    window_labels = []
    wake_labels_series = df['wake_label']
    
    for start_idx in original_indices:
        # Check if any point within the current window has a wake_label of 1
        end_idx = start_idx + window_size
        # Use .max() to check if there's any '1' (wake) in the window
        if wake_labels_series.iloc[start_idx:end_idx].max() == 1:
            window_labels.append(1) # This window contains a wake
        else:
            window_labels.append(0) # This window contains only normal points
    return np.array(window_labels)


In [15]:
# reconstructing predicted signals.
import numpy as np

def reconstruct_series_from_windows(
    original_len: int,
    original_indices,            # starts OR (start,end) pairs; list/array/tuples/dicts ok
    recon_windows: np.ndarray,   # (N, L) or (N, L, C)
    feature_idx: int | None = None,
    scaler=None,
    inverse_scale: bool = False
):
    """
    Returns:
      recon_series: (original_len,) averaged reconstruction per sample
      cover_count:  (original_len,) number of windows contributing per sample
    """
    W = recon_windows

    # pick channel if multivariate
    if W.ndim == 3:
        if feature_idx is None:
            feature_idx = 0
        W = W[:, :, feature_idx]
    elif W.ndim != 2:
        raise ValueError(f"Unexpected recon_windows shape {recon_windows.shape}; expected (N,L) or (N,L,C)")

    N, L = W.shape

    # Normalize original_indices into a list of (start, end) pairs
    pairs = []
    orig = original_indices

    # numpy array?
    if isinstance(orig, np.ndarray):
        if orig.ndim == 1:
            # 1-D starts
            for s in orig.tolist():
                s = int(s)
                pairs.append((s, s + L - 1))
        elif orig.ndim == 2 and orig.shape[1] >= 2:
            # (N,2) -> (start,end)
            for row in orig.tolist():
                pairs.append((int(row[0]), int(row[1])))
        else:
            # fallback: treat as starts
            for s in orig.reshape(-1).tolist():
                pairs.append((int(s), int(s) + L - 1))
    else:
        # list-like
        for item in list(orig):
            if isinstance(item, dict):
                s = int(item.get("start", item.get("s", 0)))
                e = int(item.get("end", item.get("e", s + L - 1)))
                pairs.append((s, e))
            elif isinstance(item, (tuple, list)) and len(item) >= 2:
                s, e = int(item[0]), int(item[1])
                pairs.append((s, e))
            else:
                # assume it's a start
                s = int(item)
                pairs.append((s, s + L - 1))

    # Optionally inverse-scale the windows (for plotting in meters)
    if inverse_scale and scaler is not None:
        W_flat = W.reshape(-1, 1)               # assumes 1 feature for plotting
        W_inv = scaler.inverse_transform(W_flat)
        W = W_inv.reshape(N, L)

    recon_series = np.zeros((original_len,), dtype=np.float32)
    cover_count  = np.zeros((original_len,), dtype=np.int32)

    for (s, e), w in zip(pairs, W):
        # clamp to series bounds
        s_clamp = max(0, min(int(s), original_len - 1))
        e_clamp = max(0, min(int(e), original_len - 1))
        if e_clamp < s_clamp:
            continue
        span = e_clamp - s_clamp + 1
        # align window length with span (handles off-by-ones)
        span = min(span, len(w))
        recon_series[s_clamp:s_clamp + span] += w[:span]
        cover_count[s_clamp:s_clamp + span]  += 1

    mask = cover_count > 0
    recon_series[mask] = recon_series[mask] / cover_count[mask]
    return recon_series, cover_count


In [16]:
# ________________________ main _________________________

In [17]:
dataset_splits = {
    'train': [], 
    'valid': [], 
    'test': []   
}

for split_name in dataset_splits.keys():
    split_path = os.path.join(base_data_dir, split_name)
    if os.path.exists(split_path):
        dataset_splits[split_name] = glob.glob(os.path.join(split_path, '*.csv'))
    else:
        print(f"Warning: Directory '{split_path}' not found. Skipping {split_name} split.")
        dataset_splits[split_name] = [] 


print("--- Starting ML Model Training and Evaluation ---")

--- Starting ML Model Training and Evaluation ---


In [18]:
# to plot a batch sample
import matplotlib.pyplot as plt
import os

PLOT_OUT_DIR = "detected_wakes_LSTM_AE/normal_window_plots_LSTM_batch"
os.makedirs(PLOT_OUT_DIR, exist_ok=True)

In [19]:
# --- Step 1: Prepare Training Data (Normal sequences only from 'train' split) ---
print("\nPreparing training data (normal sequences) for LSTM Autoencoder...")
all_train_normal_sequences_np = []
data_scaler = None 

if not dataset_splits['train']:
    print("Error: No training files found. Cannot train the model. Please check 'processed_ts/train' directory.")
else:
    for file_path in tqdm(dataset_splits['train'], desc="Processing training files"):
        df = load_and_preprocess_data(file_path)
        if df is not None:
            # Fit scaler on first file, then transform others
            if data_scaler is None:
                sequences, original_indices, data_scaler = create_sequences_pytorch(df, WINDOW_SIZE, STRIDE, fit_scaler=True)
            else:
                sequences, original_indices, _ = create_sequences_pytorch(df, WINDOW_SIZE, STRIDE, scaler=data_scaler, fit_scaler=False)


            window_labels = get_wake_labels_for_windows(df, original_indices, WINDOW_SIZE)

            normal_sequences_in_file = sequences[window_labels == 0]
            if len(normal_sequences_in_file) > 0:
                all_train_normal_sequences_np.append(normal_sequences_in_file)

    if all_train_normal_sequences_np:
        all_train_normal_sequences_np = np.concatenate(all_train_normal_sequences_np, axis=0)
        print(f"Total normal sequences for training: {all_train_normal_sequences_np.shape[0]}")
    else:
        print("No normal sequences found for training. Model will not be trained.")


Preparing training data (normal sequences) for LSTM Autoencoder...


Processing training files:   0%|          | 0/9909 [00:00<?, ?it/s]

Total normal sequences for training: 3611548


In [20]:
# --- Step 2: Build and Train LSTM Autoencoder ---
lstm_autoencoder = None
# Corrected check: ensure all_train_normal_sequences_np is a NumPy array and has elements
if all_train_normal_sequences_np is not None and all_train_normal_sequences_np.shape[0] > 0: 
    print("\nBuilding and training LSTM Autoencoder...")
    input_features = all_train_normal_sequences_np.shape[2] 
    lstm_autoencoder = LSTMAutoencoder(input_features, window_size=WINDOW_SIZE).to(device)

    optimizer = optim.Adam(lstm_autoencoder.parameters())
    criterion = nn.MSELoss() 

    train_dataset = TimeSeriesDataset(all_train_normal_sequences_np, 
                                      np.arange(len(all_train_normal_sequences_np)))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Define path for saving the best model
    os.makedirs(model_save_dir, exist_ok=True)
    best_model_path = os.path.join(model_save_dir, 'best_lstm_autoencoder.pth')

    best_loss = float('inf') # Initialize best_loss to infinity

    # Add tqdm to the epoch loop for better visibility
    for epoch in tqdm(range(EPOCHS), desc="Training Autoencoder Epochs"):
        lstm_autoencoder.train() 
        train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device) 

            optimizer.zero_grad() 
            outputs = lstm_autoencoder(data) 
            loss = criterion(outputs, data) 
            loss.backward() 
            optimizer.step() 

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Check if current model is the best
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            torch.save(lstm_autoencoder.state_dict(), best_model_path)
            tqdm.write(f"Epoch {epoch+1}/{EPOCHS}, Training Loss: {avg_train_loss:.6f} (New best model saved!)")
        else:
            tqdm.write(f"Epoch {epoch+1}/{EPOCHS}, Training Loss: {avg_train_loss:.6f}")

    print("LSTM Autoencoder training complete.")
else:
    print("Skipping LSTM Autoencoder training due to insufficient normal data.")

os.makedirs(os.path.join(output_base_dir_ml, 'train'), exist_ok=True) 
os.makedirs(os.path.join(output_base_dir_ml, 'valid'), exist_ok=True)
os.makedirs(os.path.join(output_base_dir_ml, 'test'), exist_ok=True)
os.makedirs(plots_output_dir_ml, exist_ok=True)



Building and training LSTM Autoencoder...


Training Autoencoder Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50, Training Loss: 0.035123 (New best model saved!)
Epoch 2/50, Training Loss: 0.034856 (New best model saved!)
Epoch 3/50, Training Loss: 0.034554 (New best model saved!)
Epoch 4/50, Training Loss: 0.034909
Epoch 5/50, Training Loss: 0.032608 (New best model saved!)
Epoch 6/50, Training Loss: 0.028787 (New best model saved!)
Epoch 7/50, Training Loss: 0.031763
Epoch 8/50, Training Loss: 0.028859
Epoch 9/50, Training Loss: 0.032883
Epoch 10/50, Training Loss: 0.028301 (New best model saved!)
Epoch 11/50, Training Loss: 0.028731
Epoch 12/50, Training Loss: 0.025714 (New best model saved!)
Epoch 13/50, Training Loss: 0.031437
Epoch 14/50, Training Loss: 0.028402
Epoch 15/50, Training Loss: 0.025063 (New best model saved!)
Epoch 16/50, Training Loss: 0.021327 (New best model saved!)
Epoch 17/50, Training Loss: 0.017832 (New best model saved!)
Epoch 18/50, Training Loss: 0.018139
Epoch 19/50, Training Loss: 0.021772
Epoch 20/50, Training Loss: 0.018443
Epoch 21/50, Training Loss: 0

In [20]:
# load model
best_model_path = model_save_dir + "/best_lstm_autoencoder.pth"  # detected_wakes_LSTM_AE/saved_models/w300,s5,th90
input_features = 1 # Assuming z_m is the only feature
lstm_autoencoder = LSTMAutoencoder(input_features, window_size=WINDOW_SIZE).to(device)

lstm_autoencoder.load_state_dict(torch.load(best_model_path, map_location=device))
lstm_autoencoder.eval() # Set to evaluation mode after loading
print("Model loaded successfully.")

Model loaded successfully.


In [21]:
# --- Step 3: Anomaly Threshold Determination (from 'valid' set's normal data) ---
anomaly_threshold = 0.0 
valid_normal_errors = []

if lstm_autoencoder and dataset_splits['valid'] and data_scaler:
    print("\nDetermining anomaly threshold from validation set normal data...")
    lstm_autoencoder.eval() # Set model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation for inference
        for file_path in tqdm(dataset_splits['valid'], desc="Analyzing validation files for threshold"):
            df_valid = load_and_preprocess_data(file_path)
            if df_valid is not None:
                valid_sequences_np, valid_original_indices_np, _ = create_sequences_pytorch(df_valid, WINDOW_SIZE, STRIDE, scaler=data_scaler, fit_scaler=False)
                valid_window_labels = get_wake_labels_for_windows(df_valid, valid_original_indices_np, WINDOW_SIZE)

                if len(valid_sequences_np) > 0:
                    valid_sequences_torch = torch.tensor(valid_sequences_np, dtype=torch.float32).to(device)
                    reconstructions_torch = lstm_autoencoder(valid_sequences_torch)

                    # Calculate MSE reconstruction errors (NumPy conversion for mean_squared_error)
                    errors = np.array([
                        mean_squared_error(valid_sequences_np[i].flatten(), 
                                           reconstructions_torch[i].cpu().numpy().flatten())
                        for i in range(len(valid_sequences_np))
                    ])

                    valid_normal_errors.extend(errors[valid_window_labels == 0])

        if valid_normal_errors:
            anomaly_threshold = np.quantile(valid_normal_errors, ANOMALY_THRESHOLD_QUANTILE)
            print(f"Determined Anomaly Threshold (Q{int(ANOMALY_THRESHOLD_QUANTILE*100)} on normal validation errors): {anomaly_threshold:.6f}")
        else:
            print("No normal data found in validation set to determine anomaly threshold. Using default 0.0.")
elif not lstm_autoencoder:
    print("Skipping anomaly threshold determination because the LSTM Autoencoder was not trained.")
elif not data_scaler:
    print("Skipping anomaly threshold determination because the data scaler was not fitted.")



Determining anomaly threshold from validation set normal data...


Analyzing validation files for threshold:   0%|          | 0/3411 [00:00<?, ?it/s]

Determined Anomaly Threshold (Q70 on normal validation errors): 0.007237


In [22]:
from torch.cuda.amp import autocast
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r"`torch\.cuda\.amp\.autocast\(.*\)` is deprecated"
)

In [23]:
# --- Step 4: Predict and Evaluate on Train, Validation, and Test Sets ---
overall_ground_truth_wakes_ml = []
overall_predicted_wakes_ml = []

for split_name in dataset_splits.keys():
    file_paths = dataset_splits[split_name]
    if not file_paths or not lstm_autoencoder or not data_scaler:
        print(f"\nSkipping {split_name.upper()} evaluation due to no files, model not trained, or scaler not fitted.")
        continue

    print(f"\n--- Processing {split_name.upper()} Set ({len(file_paths)} files) for ML prediction ---")

    split_ground_truth_wakes = []
    split_predicted_wakes = []
    window_level_counts = { "TP": 0, "FP": 0, "FN": 0, "TN": 0 }

    lstm_autoencoder.eval() # Set model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation
        for file_path in tqdm(file_paths, desc=f"Predicting on {split_name} files"):
            df = load_and_preprocess_data(file_path)
            if df is not None:
                sequences_to_predict_np, original_indices_predict_np, _ = create_sequences_pytorch(
                    df, WINDOW_SIZE, STRIDE, scaler=data_scaler, fit_scaler=False
                )

                if len(sequences_to_predict_np) > 0:
                    sequences_to_predict_torch = torch.tensor(sequences_to_predict_np, dtype=torch.float32).to(device)
                    reconstructions_torch = lstm_autoencoder(sequences_to_predict_torch)

#                     #Calculate errors using numpy after moving to CPU
#                     errors_predict = np.array([
#                         mean_squared_error(sequences_to_predict_np[i].flatten(), 
#                                            reconstructions_torch[i].cpu().numpy().flatten())
#                         for i in range(len(sequences_to_predict_np))
#                     ])

                    ####### AFTER (fast, torch, batched)
                    seqs = torch.as_tensor(sequences_to_predict_np, dtype=torch.float32, device=device)

                    BATCH = 4096 # tune for your GPU RAM; 1024–8192 usually fine for 1D windows
                    errors_predict = np.empty((len(seqs),), dtype=np.float32)

                    lstm_autoencoder.eval()
                    with torch.no_grad():
                        # mixed precision can speed up on GPUs
                        try:
                            use_amp = (device.type == "cuda")
                        except Exception:
                            use_amp = False
                            autocast = lambda enabled: contextlib.nullcontext()

                        for i in range(0, len(seqs), BATCH):
                            batch = seqs[i:i+BATCH]
                            with autocast(enabled=use_amp):
                                recon = lstm_autoencoder(batch)
                            diff = batch - recon
                            if diff.ndim == 3:
                                # (N, L, C)
                                batch_err = (diff ** 2).mean(dim=(1, 2))
                            else:
                                # (N, L)
                                batch_err = (diff ** 2).mean(dim=1)

                            errors_predict[i:i+BATCH] = batch_err.detach().cpu().numpy()
                    ###########
                    
                    # **** WINDOW-LEVEL metrics 
                    gt_win = get_wake_labels_for_windows(df, original_indices_predict_np, WINDOW_SIZE)
                    pred_win = (errors_predict >= anomaly_threshold).astype(np.int32)
                    
                    # Confusion counts (window-level)
                    TP_w = int(((gt_win == 1) & (pred_win == 1)).sum())
                    FP_w = int(((gt_win == 0) & (pred_win == 1)).sum())
                    FN_w = int(((gt_win == 1) & (pred_win == 0)).sum())
                    TN_w = int(((gt_win == 0) & (pred_win == 0)).sum())
                    
                    window_level_counts["TP"] += TP_w
                    window_level_counts["FP"] += FP_w
                    window_level_counts["FN"] += FN_w
                    window_level_counts["TN"] += TN_w
                    # ****

                    # EVENT-LEVEL intervals (post-processing)
                    predicted_wake_intervals, df_with_predictions = \
                        get_predicted_wake_intervals_from_errors(df.copy(), original_indices_predict_np, 
                                                                errors_predict, anomaly_threshold, 
                                                                WINDOW_SIZE, MERGE_GAP_SECONDS, MIN_WAKE_DURATION_SECONDS)

                    gt_wakes = get_ground_truth_wakes(df)
                    split_ground_truth_wakes.extend(gt_wakes)
                    split_predicted_wakes.extend(predicted_wake_intervals)

                    relative_path = os.path.relpath(file_path, base_data_dir)
                    output_file_path = os.path.join(output_base_dir_ml, relative_path)
                    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
                    
                    ####3 for ploting the predictions as well
                    recon_np = reconstructions_torch.detach().cpu().numpy()

                    # build per-sample reconstruction
                    recon_series, cover = reconstruct_series_from_windows(
                        original_len=len(df),
                        original_indices=original_indices_predict_np,
                        recon_windows=recon_np,
                        feature_idx=0,
                        scaler=data_scaler,
                        inverse_scale=True
                    )
                    recon_series_filled = recon_series.copy()
                    holes = (cover == 0)
                    if holes.any():
                        recon_series_filled[holes] = df["z_m"].values[holes]

                    df_with_predictions["z_m_recon"] = recon_series_filled

                    #######
                    
                    
                    df_with_predictions.to_csv(output_file_path, index=False)
                else:
                    print(f"\n  No sequences to predict for {file_path}. Skipping.")
            else:
                print(f"\n  Skipping processing for file: {file_path} due to load error.")

    if split_ground_truth_wakes or split_predicted_wakes:
        tp, fp, fn, accuracy, precision, recall, f1_score = evaluate_detection(
            split_ground_truth_wakes, split_predicted_wakes
        )

        print(f"\n--- ML {split_name.upper()} Evaluation Results ---")
        print(f"  Total Ground Truth Wakes: {len(split_ground_truth_wakes)}")
        print(f"  Total Predicted Wakes: {len(split_predicted_wakes)}")
        print(f"  True Positives (TP): {tp}")
        print(f"  False Positives (FP): {fp}")
        print(f"  False Negatives (FN): {fn}")
        print(f"  Accuracy (TP / Total GT): {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1_score:.4f}")
        
        print(f"_______Window level_________")
        TP = window_level_counts["TP"]
        FP = window_level_counts["FP"]
        FN = window_level_counts["FN"]
        TN = window_level_counts["TN"]
        prec_w = TP / (TP + FP) if (TP + FP) else 0.0
        rec_w  = TP / (TP + FN) if (TP + FN) else 0.0
        f1_w   = 2*prec_w*rec_w / (prec_w + rec_w) if (prec_w + rec_w) else 0.0
        acc_w  = (TP + TN) / len(gt_win) if len(gt_win) else 0.0
        print(f"  True Positives (TP): {TP}")
        print(f"  False Positives (FP): {FP}")
        print(f"  False Negatives (FN): {FN}")
        print(f"  True Negatives (TN): {TN}")
        print(f"  Accuracy (window): {acc_w:.4f}")
        print(f"  Precision: {prec_w:.4f}")
        print(f"  Recall: {rec_w:.4f}")
        print(f"  F1-Score: {f1_w:.4f}")
        
        overall_ground_truth_wakes_ml.extend(split_ground_truth_wakes)
        overall_predicted_wakes_ml.extend(split_predicted_wakes)
    else:
        print(f"  No ground truth or predicted wakes found in {split_name} set for ML evaluation.")

if overall_ground_truth_wakes_ml or overall_predicted_wakes_ml:
    tp_overall, fp_overall, fn_overall, accuracy_overall, precision_overall, recall_overall, f1_score_overall = evaluate_detection(
        overall_ground_truth_wakes_ml, overall_predicted_wakes_ml
    )

    print("\n--- Overall ML Dataset Evaluation Results ---")
    print(f"Total Ground Truth Wakes: {len(overall_ground_truth_wakes_ml)}")
    print(f"Total Predicted Wakes: {len(overall_predicted_wakes_ml)}")
    print(f"True Positives (TP): {tp_overall}")
    print(f"False Positives (FP): {fp_overall}")
    print(f"False Negatives (FN): {fn_overall}")
    print(f"Accuracy (TP / Total GT): {accuracy_overall:.4f}")
    print(f"Precision: {precision_overall:.4f}")
    print(f"Recall: {recall_overall:.4f}")
    print(f"F1-Score: {f1_score_overall:.4f}")
else:
    print("\nNo ground truth or predicted wakes found across the entire dataset for overall ML evaluation.")



--- Processing TRAIN Set (9909 files) for ML prediction ---


Predicting on train files:   0%|          | 0/9909 [00:00<?, ?it/s]


--- ML TRAIN Evaluation Results ---
  Total Ground Truth Wakes: 9672
  Total Predicted Wakes: 13773
  True Positives (TP): 8593
  False Positives (FP): 5180
  False Negatives (FN): 1079
  Accuracy (TP / Total GT): 0.8884
  Precision: 0.6239
  Recall: 0.8884
  F1-Score: 0.7330
_______Window level_________
  True Positives (TP): 670799
  False Positives (FP): 1101510
  False Negatives (FN): 1307660
  True Negatives (TN): 2510038
  Accuracy (window): 5590.2232
  Precision: 0.3785
  Recall: 0.3391
  F1-Score: 0.3577

--- Processing VALID Set (3411 files) for ML prediction ---


Predicting on valid files:   0%|          | 0/3411 [00:00<?, ?it/s]


--- ML VALID Evaluation Results ---
  Total Ground Truth Wakes: 3332
  Total Predicted Wakes: 4879
  True Positives (TP): 2993
  False Positives (FP): 1886
  False Negatives (FN): 339
  Accuracy (TP / Total GT): 0.8983
  Precision: 0.6134
  Recall: 0.8983
  F1-Score: 0.7290
_______Window level_________
  True Positives (TP): 222323
  False Positives (FP): 374791
  False Negatives (FN): 462663
  True Negatives (TN): 874159
  Accuracy (window): 1965.0215
  Precision: 0.3723
  Recall: 0.3246
  F1-Score: 0.3468

--- Processing TEST Set (6121 files) for ML prediction ---


Predicting on test files:   0%|          | 0/6121 [00:00<?, ?it/s]


--- ML TEST Evaluation Results ---
  Total Ground Truth Wakes: 3345
  Total Predicted Wakes: 8145
  True Positives (TP): 3312
  False Positives (FP): 4833
  False Negatives (FN): 33
  Accuracy (TP / Total GT): 0.9901
  Precision: 0.4066
  Recall: 0.9901
  F1-Score: 0.5765
_______Window level_________
  True Positives (TP): 225247
  False Positives (FP): 1270687
  False Negatives (FN): 458087
  True Negatives (TN): 1526388
  Accuracy (window): 3139.1308
  Precision: 0.1506
  Recall: 0.3296
  F1-Score: 0.2067

--- Overall ML Dataset Evaluation Results ---
Total Ground Truth Wakes: 16349
Total Predicted Wakes: 26797
True Positives (TP): 15045
False Positives (FP): 11752
False Negatives (FN): 1304
Accuracy (TP / Total GT): 0.9202
Precision: 0.5614
Recall: 0.9202
F1-Score: 0.6974


In [24]:
data_scaler

,copy,True
,with_mean,True
,with_std,True


In [25]:
# Inverse transform both original and reconstructed columns if they are scaled
df_with_predictions[["z_m_recon"]] = data_scaler.inverse_transform(
    df_with_predictions[["z_m_recon"]]
)

In [26]:
# --- Plotting 10 Random Validation Files for ML Model ---
if lstm_autoencoder and dataset_splits['valid'] and data_scaler:
    print("\n--- Generating Plots for 10 Random Validation Files (ML Model) ---")

    valid_files_processed_path = os.path.join(output_base_dir_ml, 'valid')
    if not os.path.exists(valid_files_processed_path):
        print(f"Error: Validation output directory '{valid_files_processed_path}' not found. Cannot generate plots.")
    else:
        validation_output_files = glob.glob(os.path.join(valid_files_processed_path, '*.csv'))

        if len(validation_output_files) == 0:
            print(f"No processed files found in '{valid_files_processed_path}'. Cannot generate plots.")
        else:
            num_plots_to_generate = min(10, len(validation_output_files))
            files_to_plot = random.sample(validation_output_files, num_plots_to_generate)

            os.makedirs(plots_output_dir_ml, exist_ok=True)

            for file_path in tqdm(files_to_plot, desc="Generating ML plots"):
                try:
                    df_to_plot = pd.read_csv(file_path)
                    plot_file_name = os.path.basename(file_path)
                    plot_wake_detection(df_to_plot, plot_file_name, save_dir=plots_output_dir_ml)
                except Exception as e:
                    print(f"Error plotting {file_path}: {e}")
            print(f"ML Plots saved to: {plots_output_dir_ml}")
else:
    print("\nSkipping ML plotting as the LSTM Autoencoder was not trained or data not processed.")




--- Generating Plots for 10 Random Validation Files (ML Model) ---


Generating ML plots:   0%|          | 0/10 [00:00<?, ?it/s]

ML Plots saved to: detected_wakes_LSTM_AE/wake_plots/w150,s5,th70
